In [1]:
# Importing aux libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
import gempy as gp

WARN: Could not locate executable g77
WARN: Could not locate executable f77
WARN: Could not locate executable ifort
WARN: Could not locate executable ifl
WARN: Could not locate executable f90
WARN: Could not locate executable efl


c:\ProgramData\anaconda3\envs\gempy1\lib\site-packages\numpy\distutils\system_info.py:2159: UserWarning: 
    Optimized (vendor) Blas libraries are not found.
    Falls back to netlib Blas library which has worse performance.
    A better performance should be easily gained by switching
    Blas library.
  if self._calc_info(blas):
c:\ProgramData\anaconda3\envs\gempy1\lib\site-packages\numpy\distutils\system_info.py:2159: UserWarning: 
    Blas (http://www.netlib.org/blas/) libraries not found.
    Directories to search for the libraries can be specified in the
    numpy/distutils/site.cfg file (section [blas]) or by setting
    the BLAS environment variable.
  if self._calc_info(blas):
c:\ProgramData\anaconda3\envs\gempy1\lib\site-packages\numpy\distutils\system_info.py:2159: UserWarning: 
    Blas (http://www.netlib.org/blas/) sources not found.
    Directories to search for the sources can be specified in the
    numpy/distutils/site.cfg file (section [blas_src]) or by setting
    t

In [3]:
geo_model = gp.create_model('Perarolo_3')

In [4]:
data_path = 'C:/Users/admin/Documents/Dottorato/Conda_envs/gempy_Rousseau/Data/'
# Importing the data from CSV-files and setting extent and resolution
gp.init_data(geo_model, [60, 220., -60., 60., 0, 140.], [50, 50, 50],
             path_i=data_path + "layer_i_nf.csv",
             path_o=data_path + "layer_o_nf.csv",
             default_values=True)

Active grids: ['regular']


Perarolo_3  2024-05-15 13:57

In [5]:
gp.map_stack_to_surfaces(geo_model,
                         {#"Fault_Series": 'Main_Fault',
                          "Strat_Series": ('gravely_layer', 'clay_layer',
                                           'altered_bedrock')},
                         remove_unused_series=True)
#"Fault_Series": 'Main_Fault',

,surface,series,order_surfaces,color,id
0,gravely_layer,Strat_Series,1,#015482,1
1,clay_layer,Strat_Series,2,#9f0052,2
2,altered_bedrock,Strat_Series,3,#ffbe00,3
3,basement,Basement,1,#728f02,4


In [6]:
gp.set_interpolator(geo_model,          
                    compile_theano=False,
                    theano_optimizer='fast_compile',
                    )

#with compile_theano=True it's really slow

Setting kriging parameters to their default values.
Compiling aesara function...
Level of Optimization:  fast_compile
Device:  cpu
Precision:  float64
Number of faults:  0
Compilation Done!
Kriging values: 
                       values
range             244.131112
$C_o$            1419.047619
drift equations       [3, 3]


In [9]:
# fp = data_path+ "topo_1_extrap.tif"
# fp

In [10]:

# geo_model.set_topography(source='gdal', filepath=fp)
# sol = gp.compute_model(geo_model, compute_mesh=True)

In [7]:
#NO TOPOGRAPHY
sol = gp.compute_model(geo_model, compute_mesh=True)

In [8]:
ver, sim = gp.get_surfaces(geo_model)
gpv = gp.plot_3d(geo_model, plotter_type='basic', show_surfaces=True, show_data=True, image=False,
                 show_topography=True)

c:\ProgramData\anaconda3\envs\gempy1\lib\site-packages\pyvista\utilities\helpers.py:507: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


In [12]:
save_path = 'C:/Users/admin/Documents/Dottorato/Conda_envs/FLAC3D_test_github/Model_surface'
geo_model.save_model(path = save_path, name='Model_surface')

True

In [48]:
geo_model = gp.load_model(path = save_path, name = 'Model_surface')

Active grids: ['regular']


In [13]:
#Try to export the surface with W6 block type
path=data_path
filename='geomodel_topo_10.f3grid'

def __build_vertices_elements_groups__2(geo_model_2):

    # get model information
    nx, ny, nz = geo_model.grid.regular_grid.resolution
    xmin, xmax, ymin, ymax, zmin, zmax = geo_model.solutions.grid.regular_grid.extent

    # create vertices array
    dx, dy, dz = (xmax - xmin) / nx, (ymax - ymin) / ny, (zmax - zmin) / nz
    n_vertices = (nx + 1) * (ny + 1) * (nz + 1)
    vertices = np.zeros((n_vertices, 3), dtype='f8')
    vertices_ids = np.arange(n_vertices)  # used to generate coordinate
    vertices[:, 0] = vertices_ids % (nx + 1) * dx + xmin
    vertices[:, 1] = (vertices_ids % ((nx + 1) * (ny + 1))) // (nx + 1) * dy + ymin
    vertices[:, 2] = vertices_ids // ((nx + 1) * (ny + 1)) * dz + zmin

    # build groups
    lith_ids = np.round(geo_model.solutions.lith_block)
    lith_ids = lith_ids.astype(int)
    sids = dict(zip(geo_model._surfaces.df['surface'], geo_model._surfaces.df['id']))
    groups = {}
    for region_name, region_id in sids.items():
        cell_ids = np.where(lith_ids == region_id)[0] + 1
        if not len(cell_ids): continue
        groups[region_name] = cell_ids

    # build elements
    n_elements = nx * ny * nz
    element_ids = np.arange(n_elements)  # used to generate elems
    elements = np.zeros((n_elements, 9), dtype='i8')
    i = element_ids % nz
    j = element_ids // nz % ny
    k = element_ids // (nz * ny)
    elements[:, 0] = 8  # all hex
    elements[:, 1] = 1 + i * (nx + 1) * (ny + 1) + j * (nx + 1) + k
    elements[:, 2] = elements[:, 1] + 1
    elements[:, 5] = elements[:, 2] + (nx + 1)
    elements[:, 3] = elements[:, 5] - 1
    elements[:, 4] = elements[:, 1] + ((nx + 1) * (ny + 1))
    elements[:, 7] = elements[:, 4] + 1
    elements[:, 8] = elements[:, 7] + (nx + 1)
    elements[:, 6] = elements[:, 8] - 1

     # remove element above topography
    #mask_topo = geo_model._grid.regular_grid.mask_topo
    shape = geo_model._grid.regular_grid.resolution
    shape_tot = shape[0]*shape[1]*shape[2]
    mask_topo = geo_model._grid.regular_grid.mask_topo
    if(mask_topo.size > 0):
        inactive_cells = mask_topo.reshape(shape_tot)
    else:
        inactive_cells = None
    if np.any(inactive_cells):
        # update correspondance
        new_id_vertices = np.zeros(len(vertices), dtype='i8')
        new_id_elements = np.zeros(len(elements), dtype='i8')
        # remove inactive cell
        elements = elements[~inactive_cells]
        new_id_elements[~inactive_cells] = np.arange(len(elements)) + 1
        # remove deleted vertices
        cond = np.isin(np.arange(len(vertices)) + 1, elements[:, 1:].flatten())
        vertices = vertices[cond]
        new_id_vertices[cond] = np.arange(len(vertices)) + 1
        # renumber vertices in element
        elements[1:] = new_id_vertices[elements[1:] - 1]
        # renumber groups
        for grp in groups.values():
            grp[:] = new_id_elements[grp - 1]

    return vertices, elements, groups

# create vertices and elements
vertices, elements, groups = __build_vertices_elements_groups__2(geo_model)

# open output file
# if not path:
#    path = './'
# if not os.path.exists(path):
#    os.makedirs(path)

out = open(path + filename, 'w')

# write gridpoints
out.write("*GRIDPOINTS")
for i, vertice in enumerate(vertices):
    out.write(f"\nG {i + 1} {vertice[0]} {vertice[1]} {vertice[2]}")

#write elements
out.write('\n*ZONES')
for i,elem in enumerate(elements):
    out.write('\nB8')
    for x in elem[1:]:
        out.write(f" {x}")

# make groups
out.write('\n*GROUPS\n')
for grp_name, grp in groups.items():
    out.write(f'ZGROUP \"{grp_name}\"\n')
    count = 0
    for x in grp:
        out.write(f"{x} ")
        count += 1
        if count == 8:
            out.write("\n")
            count = 0
    if count != 0: out.write("\n")

out.close()
print("Successfully exported geological model as FLAC3D input to " + path)


Successfully exported geological model as FLAC3D input to C:/Users/admin/Documents/Dottorato/Conda_envs/gempy_Rousseau/Data/


In [14]:
import pandas as pd

data = {'X': vertices[:,0],
        'Y': vertices[:,1],
        'Z': vertices[:,2]
}

df = pd.DataFrame(data)
df_sort = df.sort_values(by = ['X','Y','Z'])
print(df_sort)

tfi = df.groupby(['X','Y'])['Z'].transform('max') == df['Z']
tfi_i = np.invert(tfi)
df_2 = df[tfi_i]
tfi_2 = df_2.groupby(['X','Y'])['Z'].transform('max') == df_2['Z']

print(df[tfi])
print(df_2[tfi_2])

            X     Y      Z
0        60.0 -60.0    0.0
2601     60.0 -60.0    2.8
5202     60.0 -60.0    5.6
7803     60.0 -60.0    8.4
10404    60.0 -60.0   11.2
...       ...   ...    ...
122246  220.0  60.0  128.8
124847  220.0  60.0  131.6
127448  220.0  60.0  134.4
130049  220.0  60.0  137.2
132650  220.0  60.0  140.0

[132651 rows x 3 columns]
            X     Y      Z
130050   60.0 -60.0  140.0
130051   63.2 -60.0  140.0
130052   66.4 -60.0  140.0
130053   69.6 -60.0  140.0
130054   72.8 -60.0  140.0
...       ...   ...    ...
132646  207.2  60.0  140.0
132647  210.4  60.0  140.0
132648  213.6  60.0  140.0
132649  216.8  60.0  140.0
132650  220.0  60.0  140.0

[2601 rows x 3 columns]
            X     Y      Z
127449   60.0 -60.0  137.2
127450   63.2 -60.0  137.2
127451   66.4 -60.0  137.2
127452   69.6 -60.0  137.2
127453   72.8 -60.0  137.2
...       ...   ...    ...
130045  207.2  60.0  137.2
130046  210.4  60.0  137.2
130047  213.6  60.0  137.2
130048  216.8  60.0  137.2
130

In [51]:
#PROVA CON INDICI ORIGINALI ROUSSEAU
    # create vertices and elements
path=data_path
filename='geomodel_topo_10.f3grid'

def __build_vertices_elements_groups__(geo_model):

    # get model information
    nx, ny, nz = geo_model.grid.regular_grid.resolution
    xmin, xmax, ymin, ymax, zmin, zmax = geo_model.solutions.grid.regular_grid.extent

    # create vertices array
    dx, dy, dz = (xmax - xmin) / nx, (ymax - ymin) / ny, (zmax - zmin) / nz
    n_vertices = (nx + 1) * (ny + 1) * (nz + 1)
    vertices = np.zeros((n_vertices, 3), dtype='f8')
    vertices_ids = np.arange(n_vertices)  # used to generate coordinate
    vertices[:, 0] = vertices_ids % (nx + 1) * dx + xmin
    vertices[:, 1] = (vertices_ids % ((nx + 1) * (ny + 1))) // (nx + 1) * dy + ymin
    vertices[:, 2] = vertices_ids // ((nx + 1) * (ny + 1)) * dz + zmin

    # build elements
    n_elements = nx * ny * nz
    element_ids = np.arange(n_elements)  # used to generate elems
    elements = np.zeros((n_elements, 9), dtype='i8')
    i = element_ids % nz
    j = element_ids // nz % ny
    k = element_ids // (nz * ny)
    elements[:, 0] = 8  # all hex
    elements[:, 1] = 1 + i * (nx + 1) * (ny + 1) + j * (nx + 1) + k
    elements[:, 2] = elements[:, 1] + 1
    elements[:, 3] = elements[:, 2] + (nx + 1)
    elements[:, 4] = elements[:, 3] - 1
    elements[:, 5] = elements[:, 1] + ((nx + 1) * (ny + 1))
    elements[:, 6] = elements[:, 5] + 1
    elements[:, 7] = elements[:, 6] + (nx + 1)
    elements[:, 8] = elements[:, 7] - 1
    
    # build groups
    lith_ids = np.round(geo_model.solutions.lith_block)
    lith_ids = lith_ids.astype(int)
    sids = dict(zip(geo_model._surfaces.df['surface'], geo_model._surfaces.df['id']))
    groups = {}
    for region_name, region_id in sids.items():
        cell_ids = np.where(lith_ids == region_id)[0] + 1
        if not len(cell_ids): continue
        groups[region_name] = cell_ids

    # remove element above topography
    #mask_topo = geo_model._grid.regular_grid.mask_topo
    shape = geo_model._grid.regular_grid.resolution
    shape_tot = shape[0]*shape[1]*shape[2]
    mask_topo = geo_model._grid.regular_grid.mask_topo
    if(mask_topo.size > 0):
        inactive_cells = mask_topo.reshape(shape_tot)
    else:
        inactive_cells = None
    if np.any(inactive_cells):
        # update correspondance
        new_id_vertices = np.zeros(len(vertices), dtype='i8')
        new_id_elements = np.zeros(len(elements), dtype='i8')
        # remove inactive cell
        elements = elements[~inactive_cells]
        new_id_elements[~inactive_cells] = np.arange(len(elements)) + 1
        # remove deleted vertices
        cond = np.isin(np.arange(len(vertices)) + 1, elements[:, 1:].flatten())
        vertices = vertices[cond]
        new_id_vertices[cond] = np.arange(len(vertices)) + 1
        # renumber vertices in element
        elements[1:] = new_id_vertices[elements[1:] - 1]
        # renumber groups
        for grp in groups.values():
            grp[:] = new_id_elements[grp - 1]

    return vertices, elements, groups


vertices, elements, groups = __build_vertices_elements_groups__(geo_model_2)

# open output file
# if not path:
#    path = './'
# if not os.path.exists(path):
#    os.makedirs(path)

out = open(path + filename, 'w')

# write gridpoints
out.write("*GRIDPOINTS")
for i, vertice in enumerate(vertices):
    out.write(f"\nG {i + 1} {vertice[0]} {vertice[1]} {vertice[2]}")

# write elements
out.write('\n*ZONES')
zone_counter = 1 

new_column_order = [1, 2, 4, 5, 3, 8, 6, 7]

for i, elem in enumerate(elements):
    out.write(f'\nZ B8 {zone_counter}')  
    for x in new_column_order:
        out.write(f" {elem[x]}")
    zone_counter += 1 

# make groups
out.write('\n*GROUPS\n')
for grp_name, grp in groups.items():
    out.write(f'ZGROUP \"{grp_name}\"\n')
    count = 0
    for x in grp:
        out.write(f"{x} ")
        count += 1
        if count == 8:
            out.write("\n")
            count = 0
    if count != 0: out.write("\n")

out.close()
print("Successfully exported geological model as FLAC3D input to " + path)

Successfully exported geological model as FLAC3D input to C:/Users/admin/Documents/Dottorato/Conda_envs/gempy_Rousseau/Data/


In [53]:

nx, ny, nz = geo_model.grid.regular_grid.resolution
n_elements = nx * ny * nz

lis_others = []
lis_z = []
lis_plane = []
lis_xx1 = []
lis_xx2 = []
lis_yy1 = []
lis_yy2 = []

#SEARCH THE SURFACES INSIDE THE MODEL
# +1        moves in Z axis
# +nz       moves in Y axis
# +(nz*ny)  moves in X axis
W8_index = []
W6_index = []
roof = {}
count=0
df=[]

new_el = copy.deepcopy(elements)
size = np.size(elements,0)
new_groups = copy.deepcopy(groups)

for x in range(0,nz*ny*nx,nz*ny):
    for y in range(x,x+nz*ny,nz):
        for z in range(y,y+nz,1):
            if z < y+nz-1:
                lis_z.append(z)
                if lith_ids[z]!=lith_ids[z+1]:
                    #roof.update({z:[lith_ids[z], lith_ids[z+1]]})
                    new_row = new_el[z,:]
                    new_el[z,7] = new_el[z,4]
                    new_el[z,8] = new_el[z,6]
                    new_row[1] = new_row[2]
                    new_row[3] = new_row[5]
                    new_el = np.vstack((new_el, new_row))
                    l = len(new_el)-1
                    for region_name, region_id in sids.items():
                        if region_id == lith_ids[z+1]:
                            aux = new_groups[region_name]
                            aux = np.append(aux, l)
                            new_groups[region_name] = aux
                    W8_index.append(z)
                    W6_index.append(l)



# print(i)
#print(np.size(i,0))
#print(np.size(lith_ids[lis_z],0))
#print(lith_ids[lis_z])
# print(roof)
# print(len(roof))
# print(len(i))
# print(len(lis_z))

KeyError: 'altered_bedrock'

In [44]:
print(new_el[W8_index])
print(new_el[W6_index])

[[    8 33815 33815 ... 36416 33865 36416]
 [    8 41618 41618 ... 44219 41668 44219]
 [    8 62426 62426 ... 65027 62476 65027]
 ...
 [    8 54570 54570 ... 57171 54620 57171]
 [    8 62373 62373 ... 64974 62423 64974]
 [    8 80580 80580 ... 83181 80630 83181]]
[[    8 33815 33815 ... 36416 33865 36416]
 [    8 41618 41618 ... 44219 41668 44219]
 [    8 62426 62426 ... 65027 62476 65027]
 ...
 [    8 54570 54570 ... 57171 54620 57171]
 [    8 62373 62373 ... 64974 62423 64974]
 [    8 80580 80580 ... 83181 80630 83181]]


In [18]:
import copy

new_el = copy.deepcopy(elements)
size = np.size(elements,0)
new_groups = copy.deepcopy(groups)
# for x in roof:
#     if roof[x] == [4, 3]:
#         i_4_3.append(x)
#     if roof[x] == [3, 2]:
#         i_3_2.append(x)
#     if roof[x] == [2, 1]:
#         i_2_1.append(x)
lithology = []

for x in roof:
    new_row = new_el[x,:]
    new_el[x,7] = new_el[x,4]
    new_el[x,8] = new_el[x,6]
    new_row[1] = new_row[2]
    new_row[3] = new_row[5]
    new_el = np.vstack((new_el, new_row))
    l = len(new_el)-1
    
    # for region_name, region_id in sids.items():
    #     if region_id == l1-1:
    #         aux = new_groups[region_name]
    #         aux = np.append(aux, l)
    #         new_groups[region_name] = aux
print(count)          

7500


In [22]:
print(roof)

{13: [4, 3], 16: [3, 2], 24: [2, 1], 63: [4, 3], 67: [3, 2], 74: [2, 1], 114: [4, 3], 117: [3, 2], 125: [2, 1], 164: [4, 3], 168: [3, 2], 175: [2, 1], 215: [4, 3], 218: [3, 2], 225: [2, 1], 265: [4, 3], 268: [3, 2], 276: [2, 1], 316: [4, 3], 319: [3, 2], 326: [2, 1], 366: [4, 3], 369: [3, 2], 376: [2, 1], 416: [4, 3], 420: [3, 2], 427: [2, 1], 467: [4, 3], 470: [3, 2], 477: [2, 1], 517: [4, 3], 520: [3, 2], 527: [2, 1], 568: [4, 3], 571: [3, 2], 578: [2, 1], 618: [4, 3], 621: [3, 2], 628: [2, 1], 668: [4, 3], 671: [3, 2], 678: [2, 1], 719: [4, 3], 722: [3, 2], 728: [2, 1], 769: [4, 3], 772: [3, 2], 779: [2, 1], 820: [4, 3], 822: [3, 2], 829: [2, 1], 870: [4, 3], 873: [3, 2], 879: [2, 1], 920: [4, 3], 923: [3, 2], 930: [2, 1], 971: [4, 3], 974: [3, 2], 980: [2, 1], 1021: [4, 3], 1024: [3, 2], 1030: [2, 1], 1072: [4, 3], 1074: [3, 2], 1080: [2, 1], 1122: [4, 3], 1125: [3, 2], 1131: [2, 1], 1172: [4, 3], 1175: [3, 2], 1181: [2, 1], 1223: [4, 3], 1225: [3, 2], 1231: [2, 1], 1273: [4, 3], 1

In [ ]:
# import sys
# import numpy
# numpy.set_printoptions(threshold=sys.maxsize)